# ARPES data processing

In [ ]:
# Import packages
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import peaks as pks
import os
import pint_xarray

# Example data
from peaks.core.utils.sample_data import ExampleData, plot_tutorial_example_figure

# Set default options
xr.set_options(cmap_sequential='Purples', keep_attrs=True)
%matplotlib inline
%config InlineBackend.figure_format='retina'

ureg = pint_xarray.unit_registry

## Binding energy and k-space conversions  
The function :class:`peaks.core.processing.k_convert`, most conveniently accessed via the accessor `.k_convert`, allows converting ARPES data to binding energy and $k$-space, with conventions automatically following the analyser type and taking care of different scan types.

### Conversion to binding energy
To convert data to binding energy, we need some reference for the Fermi level. This should be stored as a metadata entry under the `calibration` group, and usually set via the metadata method `.set_EF_correction()`. 

:::{note}Photon energy scans
For most scans, a single `EF_correction` attribute is maintained. However, for [photon energy scans](#photon-energy-scans), an additional $hv$-dependent correction is added as another data coordinate. 
:::

The following references of the Fermi level correction are currently accepted:
- fitting parameters generated by a fit to the Fermi energy of e.g. a poly gold sample, in a dictionary form of polynomial coefficients, e.g.:
```
{'c0': 105.07837338559817,
 'c1': 0.00021092006222556432,
 'c2': -6.373415730141268e-05,
 'c3': -2.3182740086084596e-07}
```
- The correct form is automatically output by fitting a reference sample using `peaks`'s in-built [Au fitting function](./5_data_fitting.ipynb#fitting-au-fermi-edge-data), which can be passed directly to `.metadata.set_EF_correction()`
- a float, resulting in a rigid shift of the Fermi level.

E.g. taking a reference poly gold scan:

In [ ]:
#Load relevant Au correction
gold = ExampleData.gold_reference2().sel(eV=slice(16.5,None), theta_par=slice(-10,10)).bin_data(theta_par=4)
gold_fit_result = gold.fit_gold()

The resulting fit is stored as an attribute of the returned fit result:

In [ ]:
gold_fit_result.EF_correction

The $E_\mathrm{F}$ correction can be set directly from the fit_result using the metadata `set_EF_correction` method:

In [ ]:
gold.metadata.set_EF_correction(gold_fit_result)

The underlying data structure defining the EF correction can be obtained via the `.get_EF_correction()` method:

In [ ]:
gold.metadata.get_EF_correction()

The EF correction can also be defined to be like another scan:

In [ ]:
# Load a dispersion
disp1 = ExampleData.dispersion2a()

# Set the Fermi correction reference from the gold scan which already has this applied
disp1.metadata.set_EF_correction_like(gold)

In [ ]:
disp1.metadata.calibration.EF_correction

It can also be passed directly to set all members of a :class:`xarray.DataTree`:

In [ ]:
# Load data into a tree
dt = xr.DataTree()
dt.add(ExampleData.gold_reference2())
dt.add(ExampleData.dispersion2a())

# Set the EF_correction
dt.metadata.set_EF_correction_like(gold)

In [ ]:
dt.Gold.data.metadata.calibration.EF_correction

### $k$-conversion
`peaks` implements momentum conversion assuming the use of a hemispherical-type analyser with an entrance slice, with or without deflectors for virtual scanning of the angular axes, following the conventions of Ishida and Shin (https://aip.scitation.org/doi/10.1063/1.5007226). See the [documentation](https://research.st-andrews.ac.uk/kinggroup/peaks/latest/explanations/angle_conventions.html) for more information and a discussion of convention choices.

#### Dispersions
To process e.g. a dispersion: 

In [ ]:
# Estimate normal emission from our dispersion
disp1.plot()
plt.axvline(3.85)

Set the normal emission values as described in the [metadata guide](./1_getting_started.ipynb#setting_manipulator_normal_emissions).

In [ ]:
disp1.metadata.set_normal_emission(theta_par=3.85*ureg.deg)

In [ ]:
disp1.metadata.manipulator

In [ ]:
disp1_k = disp1.k_convert()

In [ ]:
disp1_k.plot()
plt.axvline(0)

We can specify a desired k-point range and step size by providing the `ky` argument (note depending on the analyser, this could be `kx`) as a `slice(start, stop, step)`. Note, any of these can be retained at the default by passing `None` for that parameter:

In [ ]:
disp1_k2 = disp1.k_convert(ky=slice(-0.4,None,0.05))
pks.plot_grid([disp1_k, disp1_k2], titles=["Auto", "$\\Delta{}k=0.05\\;\\AA^{-1}$"])

:::{tip}
Specifying a reduced step size for the $k$ conversion only reduces the sampling density for the interpolation, it does not bin the data. If you are happy with a larger step size, it is better to first [bin the data](#data-binning) using e.g.:

```python
disp1.bin_data(theta_par=2)
```
:::

Note that the methods attempt to estimate unspecified parameters if they are not set - note the `Analysis warning` boxes returned. E.g. if passing a freshly loaded dispersion without setting the `EF_correction` attribute, an automatic determination of the Fermi level will be attempted: 

In [ ]:
disp1_copy = ExampleData.dispersion2a()
disp1_copy.k_convert().plot()

:::{warning}
These automatic estimations are useful for a quick look, but should not be relied upon for careful analysis!
:::

#### Fermi surfaces
Fermi surface corrections work in the same way as for dispersions.

In [ ]:
#Load a Fermi map and relevant Au correction
FS = ExampleData.FS()
gold = ExampleData.gold_reference()
# Select a smaller range
gold = gold.sel(theta_par=slice(-16,17), eV=slice(104.9,105.2)).bin_data(theta_par=5,eV=2)

In [ ]:
# Determine normal emission in FS.disp()
FS.metadata.set_normal_emission({'polar': '1.499 deg', 'tilt': '0.020999999999999998 deg', 'azi': '-12 deg'})

In [ ]:
FS.metadata.manipulator

In [ ]:
fit_result = gold.fit_gold()
FS.metadata.set_EF_correction(fit_result)

To return just a single slice in energy, the `eV_slice=(centre, width)` argument can be used, with the arguments specified in binding energy:

In [ ]:
FSM_k = FS.k_convert(eV_slice=(-0.05,0.02))
FSM_k.plot()
plt.axvline(0)
plt.axhline(0)

Or the full cube can be $k$-converted

:::{tip}
`peaks` uses :class:`numba` and :class:`numexpr` to accelerate the interpolations and relevant angle <--> $k$ calculations, making the $k$-conversions rather fast. Still, for a large map these can still be somewhat time consuming, and it is often better perform binning on the data first.
:::

In [ ]:
FS_bin = FS.bin_spectra(2)  # 2x2 bin on the eV and theta_par dimensions

In [ ]:
FS_bin_k = FS_bin.k_convert()

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(10,4))
FS_bin_k.sel(kx=0,method='nearest').plot(y='eV', ax=axes[0])
FS_bin_k.sel(ky=0,method='nearest').plot(y='eV', ax=axes[1])
plt.tight_layout()

### Photon energy scans

#### Loading hv-dep data/data format
In an $h\nu$ scan, the most natural representation of the data is as a 3d cube of ($hv$,theta_par,eV), however the eV axis in raw kinetic energy changes as a function of photon energy, adding some extra complexity.

In :class:`peaks`, the convention is to load (or assemble) a $h\nu$-dependent dataset into such a cube, where the eV coordinate (when in kinetic energy mode) is that of the first scan, while a second non-dimension coordinate is included which shows the relevant offset of the KE scale as a function of hv. A warning on load indicates this. 

In [ ]:
# Load data (kz map)
hv1 = ExampleData.hv_map()

:::{note}
Due to this perculiar data structure, if a single scan is selected using the usual `.sel(hv=...)` method, the incorrect energy scale will be returned. We have a helper method, `disp_from_hv` to overcome this problem
:::

In [ ]:
pks.plot_grid([hv1.sel(hv=70, method='nearest'),hv1.disp_from_hv(70)], titles=['from `sel`', 'from `disp_from_hv`'])

Note the incorrect energy scale on the left!

#### Calibration
##### Normal emission
The normal emission is set as normal

In [ ]:
# Check and set normal emission
hv1.disp_from_hv(70).sel(theta_par=slice(-4,5)).plot()
plt.axvline(1.05)
hv1.metadata.set_normal_emission(theta_par=1.05)

##### Inner potential
An inner potential should be set in `.metadata.calibration.V0`. If nothing is set, a default value of 12 eV is used

In [ ]:
hv1.metadata.calibration.V0 = 15 * pks.ureg('eV')

##### EF_correction
For the energy calibration, a regular `EF_correction` can be set, but also a $h\nu$-dependent shift must be included as a co-ordinate within the :class:`xarray.DataArray` itself.

We set the regular angle-dependent part like normal

In [ ]:
# Load a gold reference scan for a single energy
gold = ExampleData.gold_reference3().sel(theta_par=slice(-15,15),eV=slice(22.2,22.7))
fit_result = gold.fit_gold()

In [ ]:
# Adding the angle-dep part
hv1.metadata.set_EF_correction(fit_result)

The $h\nu$-dependent correction can be added as a new co-ordinate:

```python
data = data.coords.update({"EF": ("hv", EF_values)})
```
where `EF_values` is a :class:`numpy.ndarray` of length equal to the number of photon energies. Or it can be automatically estimated from `.estimate_EF()` which is run automatically if the `.k_convert` function is run with no Fermi correction already set.

#### $k$-conversion

In [ ]:
# Now do the k-conversion and the rest is estimated
hv1_k = hv1.k_convert()

In [ ]:
# Plot a slice through the map
hv1_k.drop_zero_borders().MDC(-0.40, 0.02).plot()
plt.axvline(0)

:::{tip}
Note the `.drop_zero_borders()` method (:class:`peaks.core.process.data_select.drop_zero_borders`)in the above - this trims any rows/columns/slices that are only zeros, useful to trim the borders. :class:`peaks.core.process.data_select.drop_nan_borders` is an equivalent function for border regions that are full of only NaNs.
:::

The other arguments for selecting the $k$ and $E$ range returned work as before, while an additional argument `return_kv_scan_in_hv` allows for the data to be returned vs. photon energy instead of $k_z$:

In [ ]:
hv1_kip = hv1.k_convert(return_kz_scan_in_hv=True)
hv1_kip.MDC(-0.40, 0.02).drop_zero_borders().plot()
plt.axvline(0)

## Merging data
Scans can be merged together using the :class:`peaks.core.process.tools.merge_data` function, avaialble as `pks.merge_data`. By default this acts along the `theta_par` dimension, but can be applied along any dimension and to higher dimensional data, with various in-built options for specifying the offsets and pre-cropping the data:

In [ ]:
#Load multiple scans
disps = xr.DataTree() 
for i, scan in enumerate(['a','b','c']):
    disps.add(getattr(ExampleData,f"dispersion2{scan}")(), name=f'Scan {i+1}')

# Merge the data, applying offsets in the default angle direction and trimming the data along this dimension 
merged_disp = pks.merge_data(disps, offsets=12, sel=slice(-10,10))

In [ ]:
disps.add(merged_disp, name='Merged scan')
disps.plot_grid(ncols=4, sharey=True)

## Normalisation
To normalise data by a float, EDC/MDC, or region of interest, use :class:`peaks.core.process.tools.norm`, available via the `.norm()` accessor. This broadcasts across any number of dimenisons. Call with no argument to normalise to the max value, normalise by the mean by passing `all`, or by an integrated DC along a specified dimension, `dim`, or pass a keyword argument with `dim=slice(start,stop)` to define the region to make a DC to align by.

In [ ]:
disp1_norm1 = disps['Scan 1'].data.norm()
disp1_norm2 = disps['Scan 1'].data.norm(eV=slice(17,17.5))
disp1_norm3 = disps['Scan 1'].data.norm('eV')
pks.plot_grid([disps['Scan 1'].data, disp1_norm1, disp1_norm2, disp1_norm3],
              titles=['Original','Normalised to max', "Normalised by MDC above $E_F$", "Normalised by integrated MDC"],
             ncols=4)

## Background subtraction
Background subtraction works in exactly the same way, except calling `.bgs`, with the relevant parameters passed definie the data to subtract, except that either the `subtraction` arguement must be passed (can be a `float` or `int` to subtract a constant), or a keyword argument must be supplied specifying the background region to subtract. In addition `subtraction="Shirley"` can be passed to specify subtraction of a Shirley background, in which case some additional arguments can be supplied to define options for the Shirley background determination. 

In [ ]:
disp1_bgs1 = disps['Scan 1'].data.bgs(5*pks.ureg('kcount/s'))
disp1_bgs2 = disps['Scan 1'].data.bgs(5000)
disp1_bgs3 = disps['Scan 1'].data.bgs(eV=slice(17,17.5))
disp1_bgs4 = disps['Scan 1'].data.bgs('theta_par')
disp1_bgs5 = disps['Scan 1'].data.bgs('Shirley')
pks.plot_grid([disps['Scan 1'].data, disp1_bgs1, disp1_bgs2, disp1_bgs3, disp1_bgs4, disp1_bgs5],
              titles=['Original','Constant bg of 5 kcount/s subtracted', 'Constant bg of 5000 subtracted', 'Subtracted MDC above $E_F$', "Subtracted integrated EDC", "Subtracted Shirley BG"],
             ncols=3,
             vmin=0,
             sharey=True,
             add_colorbar=False,
             figsize=(14,8))

## Data binning
Binning can be performed using the `.bin_data()` function, which provides a thin wrapper around the built-in :class:`xarray.coarsen` function, but with updating of analysis history built in, and with a shortcut for uniform binning. 

In [ ]:
disp = ExampleData.dispersion()
binned_disps = [disp,
         disp.bin_data(2),
         disp.bin_data(eV=8,theta_par=10, boundary='pad'),
        ]
pks.plot_grid(binned_disps,
              titles=['Original', '2x2 binning applied', '8x10 binning applied with boundary padding'],
             sharey=True,
             add_colorbar=False
             )

`.bin_spectra` can also be used to enforce binning on only the spectral dimensions of an ARPES detector (`eV`, `theta_par`, `k_par`), useful e.g. for binning of spatial mapping or Fermi surface data to avoid binning also the spatial or coarse angular dimentions. Both can be applied directly to :class:`xarray.DataTree`s 

In [ ]:
disps.bin_spectra(8).plot_grid()

## Smoothing

Smoothing of data can be applied using `peaks.core.process.tools.smooth`, or via the `.smooth` accessor, passing keyword argument(s) in the format `axis=FWHM`, where FWHM is the relevant FWHM of the Gaussian for convolution in this direction, e.g. eV=0.1. This broadcasts to multidimensional data.

In [ ]:
disp_smooth = disp.smooth(eV=10*pks.ureg('meV'),theta_par=1)
pks.plot_grid([disp, disp_smooth],
              titles=['Original','Smoothed'])

## Derivative-type methods
We have various wrappers around the built-in :class:`xarray.DataArray.differentiate` method to perform derivative methods for ARPES analysis, including updating the analysis history. `.deriv(dim=list_of_dims)` is our general method, but specific `.d2E()`, `.d2k()`, `.dEdk()`, `.dkdE()`, `.curvature()` and `.min_gradient()` methods exist to ease use of derivative methods for ARPES analsys. Note, often data must be pre-smoothed. 

:::{warning}
Note, derivative methods can introduce significant artefacts in ARPES data - use with extreme caution!
:::

In [ ]:
disp_d2k = disp.sel(theta_par=slice(-15,15)).smooth(eV=0.01,theta_par=0.2).d2k()
disp_curv = disp.sel(theta_par=slice(-15,15)).smooth(eV=0.03,theta_par=0.5).curvature(theta_par=500, eV=50)
fig, ax = plt.subplots(ncols=3, figsize=(12,4))
disp.sel(theta_par=slice(-15,15)).plot(ax=ax[0])
ax[0].set_title('Original')
disp_d2k.plot(ax=ax[1], vmax=0)
ax[0].set_title('d2/dk^2')
disp_curv.plot(ax=ax[2], robust=True)
ax[0].set_title('Curvature')
plt.tight_layout()

## Symmetrisation
The :class:`peaks.core.process.tools.sym` function (`.sym()` accessor) allows to symmetrise (or alternatively just flip) data around a given axis defined via a keyword argument to the function call.

### $E_\mathrm{F}$ symmetrisation
If no arguments are supplied, `.sym()` defults to symmetrising in energy around `eV=0`, useful e.g. for symmetrising about the Fermi level for gap analysis etc. 

In [ ]:
disp1_k.sym().plot()

In [ ]:
disp1.sym(eV=16.8*pks.ureg('eV'))

In [ ]:
disps.sym(eV=16.8).plot_grid()

This broadcasts to higher dimensions, and can be used to also symmetrise in a different dimension, but can only be applied to symmetrisiation in a single axis.

### Rotational symmetrisation
The :class:`peaks.core.process.tools.sym_nfold` function (`.sym_nfold()` accessor) allows for $n$-fold symmetrisation of a 2D :class:`xarray.DataArray` around specified centre co-ordinates (defaults to (0,0)).

In [ ]:
FSM_k_s3 = FSM_k.sym_nfold(3, kx=0.0, ky=0)
pks.plot_grid([FSM_k,FSM_k_s3], titles=['Original','3-fold symmetrised'])

## Grid removal

:::{note}
`peaks` currently has only a basic implementation of analyser detector mesh removal. For a more complete implementation see [Liu et al.](https://www.sciencedirect.com/science/article/pii/S0368204822000883)
:::

Automated mesh removal can be attempted with the `.degrid()` function

In [ ]:
# Load some data with detector mesh visible
disp2 = ExampleData.dispersion3()
disp2_dg = disp2.degrid()

:::{warning}
This feature is currently experimental, and as is clear in the above example, can introduce artefacts in the data. Use with care!
:::